In [1]:
import os
os.chdir("../")

In [2]:
from dataclasses import dataclass
from pathlib import Path
from src.exception import CustomException

In [3]:
@dataclass(frozen=True)
class BaseModelConfig:
    root_dir: str
    base_model_path: Path
    updated_base_model_path: Path
    params_img_size: list #as per VGG16 model, it s required to have the image size as 224,224,3
    params_batch_size: int
    params_include_top: bool
    params_epochs: int
    params_classes: int
    params_weights: str
    params_learning_rate: float

In [4]:
from src.utils.common import create_directories
from src.utils.common import read_yaml
from src.constants import constant_paths

CONFIG_FILE_PATH = constant_paths().get_paths()[0]
PARAMS_FILE_PATH = constant_paths().get_paths()[1]

In [5]:
class ConfigurationManager:
    def __init__(
            self,
            config_file_path = CONFIG_FILE_PATH,
            params_file_path = PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)

        #create_directories([self.config.artifacts_dir])

    def prepare_base_model(self) -> BaseModelConfig:
        config = self.config.base_model

        create_directories([config.root_dir])

        base_model_config = BaseModelConfig(
            root_dir = config.root_dir,
            base_model_path=config.base_model_path,
            updated_base_model_path= config.updated_base_model_path,
            params_img_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE,
            params_include_top=self.params.INCLUDE_TOP,
            params_learning_rate=self.params.LEARNING_RATE,
            params_epochs=self.params.EPOCHS,
            params_weights=self.params.WEIGHTS,
            params_classes=self.params.CLASSES
        )
        #returning a class with loaded data path files
        return base_model_config

Now that the conifguration is done, we need to create a prepare_base_model class


In [6]:
import os
import sys
import tensorflow as tf
from src.logger import logging
from zipfile import ZipFile
import urllib.request as request


In [7]:
class PrepareBaseModel:
    def __init__(self, config: BaseModelConfig):
        self.config = config

    @staticmethod
    def save_model(path: Path,model:tf.keras.Model):
        model.save(path)
   
    def download_base_model(self):
        try:
            self.model = tf.keras.applications.vgg16.VGG16(
                input_shape=self.config.params_img_size,
                weights = self.config.params_weights,
                include_top=self.config.params_include_top
            )
            self.save_model(path = self.config.base_model_path,model = self.model)
            logging.info(f"Base model downloaded to {self.config.base_model_path}")

        except Exception as e:
            raise CustomException(e,sys)
    
    @staticmethod
    def prepare_full_model(model,classes,freeze_all,freeze_till,learning_rate):
        if freeze_all:
            for layer in model.layers:
                model.trainable = False

        elif (freeze_till is not None) and (freeze_till >= 0):
            for layer in model.layers[:-freeze_till]:
                model.trainable = False
        
        flatten_in = tf.keras.layers.Flatten()(model.output)
        prediction = tf.keras.layers.Dense(units=classes,activation='softmax')(flatten_in)

        logging.info("the output layer is added to the model")

        full_model = tf.keras.models.Model(inputs=model.input,outputs=prediction)

        full_model.compile(
                    optimizer=tf.keras.optimizers.SGD(learning_rate=learning_rate),
                    loss=tf.keras.losses.CategoricalCrossentropy(),
                    metrics=['accuracy']
                )
        logging.info("the model is compiled")

        full_model.summary()
        return full_model
    
    def update_base_model(self):
        self.full_model = self.prepare_full_model(model=self.model,classes = self.config.params_classes,freeze_all = True,freeze_till = None,learning_rate = self.config.params_learning_rate)
        self.save_model(path = self.config.updated_base_model_path,model = self.full_model)

        logging.info(f"Updated Base model updated to {self.config.updated_base_model_path}")



In [8]:
## Pipeline

try:
    configmanager = ConfigurationManager()
    prepare_base_model_config = configmanager.prepare_base_model()
    prepare_base_model = PrepareBaseModel(prepare_base_model_config)
    prepare_base_model.download_base_model()
    prepare_base_model.update_base_model()

except Exception as e:
    raise CustomException(e,sys)

[2023-07-27 18:13:57,424] 40 root INFO - yaml file: config\config.yaml loaded successfully
[2023-07-27 18:13:57,431] 40 root INFO - yaml file: params.yaml loaded successfully
[2023-07-27 18:13:57,433] 59 root INFO - created directory at: artifacts/base_model


58889256/58889256 [==============================] - 2s 0us/step
[2023-07-27 18:13:59,719] 359 tensorflow WARNING - Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` will be empty until you train or evaluate the model.
[2023-07-27 18:13:59,827] 17 root INFO - Base model downloaded to artifacts/base_model/base_model.h5
[2023-07-27 18:13:59,847] 35 root INFO - the output layer is added to the model
[2023-07-27 18:13:59,859] 44 root INFO - the model is compiled
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
            

c:\Users\pragn\anaconda3\envs\final_CDC_env\Lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


[2023-07-27 18:14:00,008] 53 root INFO - Updated Base model updated to artifacts/base_model/base_model_updated.h5
